In [698]:
import pandas as pd
import numpy as np
from datetime import datetime 

### Function

In [699]:
def read_until_null_excel(file_path, sheet_name):
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    null_row_index = df.isnull().all(axis=1).idxmax() if df.isnull().all(axis=1).any() else None

    if null_row_index is not None:
        df = df.iloc[:null_row_index]
    return df

def is_group_in_item(item, group):
    item_str = str(item)
    group_str = str(group)
    return group_str in item_str
def map_group_numbers(df1, df2, item_column, vendor_column, groups_column):
    result = []
    for _, row in df1.iterrows():
        item = row[item_column]
        vendor = str(row[vendor_column])
        matched_group = ''
        for _, group_row in df2.iterrows():
            group_vendor = str(group_row[vendor_column])
            if group_vendor == vendor:
                group = group_row[groups_column]
                if is_group_in_item(item, group):
                    matched_group = group
                    print(f"Match found: Item '{item}' (Vendor: '{vendor}') matches Group '{group}'")
                    break
        result.append(matched_group)
        if not matched_group:
            print(f"No match for Item '{item}' (Vendor: '{vendor}')")
    return result

def first_non_null_column_name(df, start_col, end_col, new_col_name):
    columns_to_check = df.loc[:, start_col:end_col].columns

    def find_first_non_null(row):
        for col in columns_to_check:
            if pd.notnull(row[col]) and row[col] != 0:
                return col
        return None

    df[new_col_name] = df.apply(find_first_non_null, axis=1)
    return df

#Sum pairs
def sum_pairs(df):
    # Initialize an empty list to store the results
    result_list = []
    
    # Separate numeric and non-numeric columns
    numeric_cols = df.select_dtypes(include='number').columns
    non_numeric_cols = df.select_dtypes(exclude='number').columns
    
    # Iterate through the DataFrame in steps of 2
    for i in range(0, len(df), 2):
        # Check if the next row exists
        if i + 1 < len(df):
            # Sum the current row and the next row for numeric columns
            summed_row = df.iloc[i:i+2][numeric_cols].sum()
            # Keep the non-numeric column from the first row
            non_numeric = df.iloc[i][non_numeric_cols]
            # Combine numeric and non-numeric results
            combined = pd.concat([summed_row, non_numeric])
            # Append the result to the list
            result_list.append(combined)
    
    # Convert the list of results to a DataFrame
    result = pd.DataFrame(result_list)
    
    return result

def returnS_F(df_1):
    result_list = []
    for row in range(0, len(df_1), 2):
        found = False
        for col in range(0, len(df_1.columns),1):
            if df_1.iloc[row, col] > 0:
                result_list.append(df_1.columns[col])
                found = True
                break
        if not found and row + 1 < len(df_1):
                for col in range(0, len(df_1.columns),1):
                    if df_1.iloc[row + 1, col] > 0:
                        result_list.append(df_1.columns[col])
                        found = True
                        break
        if not found:
            result_list.append(" ")
    result = pd.DataFrame(result_list, columns=['New-Outcome'])
    return result

def highlight_condition_Arca_EC(df, s1, s2,date_columns_1):
    # Define a function to apply to each row
    def highlight(row):
        # Initialize a list of empty styles for each column
        styles = [''] * len(row)
        
        # Apply red background if the condition is met
        if row[s1] < row[s2]:
            styles[df.columns.get_loc(s1)] = 'background-color: red'
            styles[df.columns.get_loc(s2)] = 'background-color: red'
        
        return styles
    df[date_columns_1] = df[date_columns_1].astype(str)
    df[date_columns_1] = df[date_columns_1].replace('1999-01-01', '0')
    df['Vendor #'] = df['Vendor #'].astype(int)
    # Apply the function row-wise
    return df.style.apply(highlight, axis=1)

def parse_dates(date_str):
    try:
        return pd.to_datetime(date_str, format='%Y-%m-%d')
    except ValueError:
        return pd.NA

### Import File

In [700]:
df_2 = pd.read_csv('8.12 Production ScheduleDoLuong215041.csv', skiprows = 6)
df_2['Vendor #'] = df_2['Vendor #'].astype(float)

In [701]:
file_input = '8.12 April HP 2024 Price Confirmation Roll-Up.xlsx'
sheet_name = 'April HP 2024 Price Confirmatio'
df_1 = read_until_null_excel(file_input, sheet_name)

/Users/thaongan/Documents/Python/ToolAutomation/.venv/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


### Modified df_1 (File SmartSheet)

In [702]:
vendor_take = df_1['Vendor #'].unique()
df_2 = df_2[df_2['Vendor #'].isin(vendor_take)]

In [703]:
split_columns = df_1['Group Number'].str.split(' ', expand=True)

In [704]:
if split_columns.shape == 2:
    split_columns.columns = ['Group Number Split','Additional Component']
else:
    split_columns = split_columns.rename(columns={0: 'Group Number Split'})
    split_columns['Additional Component'] = None

In [705]:
df_1 = pd.concat([df_1, split_columns], axis=1)
group_number_list = df_1['Group Number Split'].unique().tolist()
df_1.drop(columns=['Group Number'], inplace=True)
df_1 = df_1.rename(columns={'Group Number Split': 'Group Number'})

### Modified df_2 (File System)

In [706]:
df_2['Item #'] = df_2['Item #'].str.replace('="', '').str.replace('"', '')

In [707]:
len(df_2)*2/3

13590.0

In [708]:
df_2_sample = df_2.loc[(df_2['S/F/P'] == 'S') | (df_2['S/F/P'] == 'F')]
df_2_sample = df_2_sample.loc[:,df_2_sample.columns[4]:]

In [709]:
df_2_sample  = df_2_sample.astype(float)

In [710]:
df_2_sample

,7/27,8/3,8/10,8/17,8/24,8/31,9/7,9/14,9/21,9/28,...,11/2,11/9,11/16,11/23,11/30,12/7,12/14,12/21,12/28,1/4
6,0.0,0.0,0.0,6000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,900.0,0.0,0.0,200.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30,0.0,4600.0,0.0,4600.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31,0.0,0.0,0.0,0.0,4600.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39,0.0,6000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151411,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
151413,0.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
151414,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
151416,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [711]:
df = pd.DataFrame(df_2_sample)
result = returnS_F(df)

In [712]:
len(df_2_sample)

13590

In [713]:
result

,New-Outcome
0,8/17
1,8/3
2,8/3
3,8/3
4,8/3
...,...
6790,8/17
6791,7/27
6792,
6793,8/10


In [714]:
df_2 = df_2.loc[df_2['S/F/P'] == 'F'].reset_index(drop=True)

In [715]:
# df_2.iloc[:, 4:].astype(float)
# df_2.iloc[:, 4:] = np.nan

In [716]:
# repeated_result = result.reindex(result.index.repeat(1)).reset_index(drop=True)

In [717]:
df_2['Date_First_Value'] = result

In [718]:
item_column = 'Item #'
vendor_column = 'Vendor #'
groups_column = 'Group Number'
df_2['Group Number'] = map_group_numbers(df_2, df_1, item_column, vendor_column, groups_column)

No match for Item '01134' (Vendor: '900352.0')
No match for Item '01222' (Vendor: '900352.0')
No match for Item '01235' (Vendor: '900352.0')
No match for Item '01266' (Vendor: '900352.0')
No match for Item '01279' (Vendor: '900352.0')
No match for Item '01281' (Vendor: '900352.0')
No match for Item '01282' (Vendor: '900352.0')
No match for Item '02193' (Vendor: '900352.0')
No match for Item '03034' (Vendor: '900352.0')
No match for Item '03948' (Vendor: '900352.0')
No match for Item '04483' (Vendor: '900352.0')
No match for Item '04484' (Vendor: '900352.0')
No match for Item '09646' (Vendor: '900352.0')
No match for Item '09697' (Vendor: '900352.0')
No match for Item '100-17' (Vendor: '617616.0')
No match for Item '100-18' (Vendor: '617616.0')
No match for Item '100-62' (Vendor: '617616.0')
No match for Item '100-62' (Vendor: '617616.0')
No match for Item '100-62' (Vendor: '617616.0')
No match for Item '100-62' (Vendor: '617616.0')
No match for Item '10516565' (Vendor: '900352.0')
No m

### Create File Check

In [719]:
df_check = df_2.loc[df_2['Group Number'] != '']

In [720]:
df_check.loc[:, 'Arcadia ETD System'] = np.where(df_check['Whse'].isin({'1'}), 1, 0)
df_check.loc[:, 'EC ETD System'] = np.where(df_check['Whse'].isin({'15', '17', 'ECR'}), 1, 0)
df_check.loc[:, 'WC ETD System'] = np.where(df_check['Whse'].isin({'42', '28', '5'}), 1, 0)

/var/folders/c7/zh_6w0ts24d8rh3tsybfhrz80000gn/T/ipykernel_34994/385469515.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_check.loc[:, 'Arcadia ETD System'] = np.where(df_check['Whse'].isin({'1'}), 1, 0)
/var/folders/c7/zh_6w0ts24d8rh3tsybfhrz80000gn/T/ipykernel_34994/385469515.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_check.loc[:, 'EC ETD System'] = np.where(df_check['Whse'].isin({'15', '17', 'ECR'}), 1, 0)
/var/folders/c7/zh_6w0ts24d8rh3tsybfhrz80000gn/T/ipykernel_34994/385469515.py:3

In [721]:
df_check_v2  = pd.merge(df_check, df_1[['Vendor #','Group Number','Additional Component','Arcadia ETD','EC ETD', 'WC ETD','Categories']], on =['Group Number','Vendor #'], how = 'left')

In [722]:
df_check_v3 = df_check_v2

In [723]:
current_year = datetime.now().strftime('%Y')
df_check_v3['Date_First_Value'] = df_check_v3['Date_First_Value'].str.strip()
df_check_v3['Date_First_Value']= df_check_v3['Date_First_Value'] + '/' + current_year

In [724]:
df_check_v3['Date_First_Value'] = df_check_v3['Date_First_Value'].replace('/2024','01/01/2027')
df_check_v3['Date_First_Value'] = pd.to_datetime(df_check_v3['Date_First_Value'], format='%m/%d/%Y')
df_check_v3['Date_First_Value'] = df_check_v3['Date_First_Value'].dt.strftime('%Y-%m-%d')
df_check_v3['Date_First_Value'] = df_check_v3['Date_First_Value'].replace('2027-01-01','0')

In [725]:
df_check_v3['Arcadia ETD System Final'] = np.where(df_check_v3['Arcadia ETD System'] == 1, df_check_v3['Date_First_Value'],0)
df_check_v3['EC ETD System Final'] = np.where(df_check_v3['EC ETD System'] == 1, df_check_v3['Date_First_Value'],0)
df_check_v3['WC ETD System Final'] = np.where(df_check_v3['WC ETD System'] == 1, df_check_v3['Date_First_Value'],0)

In [726]:
for col in ['Arcadia ETD', 'EC ETD', 'WC ETD']:
    df_check_v3[col] = df_check_v3[col].dt.strftime('%Y-%m-%d')

In [727]:
df_check_v3['Arcadia ETD Smartsheet'] = np.where(df_check_v3['Arcadia ETD System'] == 1, df_check_v3['Arcadia ETD'], 0)
df_check_v3['EC ETD Smartsheet'] = np.where(df_check_v3['EC ETD System'] == 1, df_check_v3['EC ETD'], 0)
df_check_v3['WC ETD Smartsheet'] = np.where(df_check_v3['WC ETD System'] == 1, df_check_v3['WC ETD'], 0)

In [728]:
df_filtered_next = df_check_v3[['Categories','Item #','Whse','Group Number','Additional Component','Vendor #','Arcadia ETD System Final','EC ETD System Final','WC ETD System Final','Arcadia ETD Smartsheet','EC ETD Smartsheet','WC ETD Smartsheet']]

In [729]:

df_filtered_next['Check True/False'] = np.where(
    (df_filtered_next['Arcadia ETD System Final'] == df_filtered_next['Arcadia ETD Smartsheet']) &
    (df_filtered_next['EC ETD System Final'] == df_filtered_next['EC ETD Smartsheet']) &
    (df_filtered_next['WC ETD System Final'] == df_filtered_next['WC ETD Smartsheet']),
    'True', 'False'
)

/var/folders/c7/zh_6w0ts24d8rh3tsybfhrz80000gn/T/ipykernel_34994/105662098.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_next['Check True/False'] = np.where(


In [730]:
df_filtered_next

,Categories,Item #,Whse,Group Number,Additional Component,Vendor #,Arcadia ETD System Final,EC ETD System Final,WC ETD System Final,Arcadia ETD Smartsheet,EC ETD Smartsheet,WC ETD Smartsheet,Check True/False
0,Accessories,A1001080,5,A1001080,None,613923.0,0,0,2024-08-03,0,0,2024-08-03,True
1,Accessories,A1001081,5,A1001081,None,613923.0,0,0,2024-09-07,0,0,2024-08-31,False
2,Accessories,A1001082,5,A1001082,None,613923.0,0,0,2024-08-03,0,0,2024-08-03,True
3,Accessories,A4000669,1,A4000669,None,646821.0,2024-08-10,0,0,2024-08-10,0,0,True
4,Accessories,A4000669,15,A4000669,None,646821.0,0,2024-08-10,0,0,2024-08-03,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1088,Wall Unit,W100-22,15,W100,None,606629.0,0,2024-08-10,0,0,NaN,0,False
1089,Wall Unit,W100-22,17,W100,None,606629.0,0,2024-07-27,0,0,NaN,0,False
1090,Wall Unit,W100-22,28,W100,None,606629.0,0,0,2024-08-03,0,0,NaN,False
1091,Wall Unit,W100-22,5,W100,None,606629.0,0,0,2024-08-03,0,0,NaN,False


In [731]:
df_filtered_next = df_filtered_next.loc[df_filtered_next['Whse'].isin({'1','15','17','ECR','42','28','5'})]

In [732]:
df_filtered_next['Additional Component'] = df_filtered_next['Additional Component'].fillna('None')

/var/folders/c7/zh_6w0ts24d8rh3tsybfhrz80000gn/T/ipykernel_34994/3438248237.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_next['Additional Component'] = df_filtered_next['Additional Component'].fillna('None')


In [733]:
date_columns = ['Arcadia ETD System Final','EC ETD System Final','WC ETD System Final','Arcadia ETD Smartsheet','EC ETD Smartsheet','WC ETD Smartsheet']
df_filtered_next[date_columns] = df_filtered_next[date_columns].astype(str)
df_filtered_next[date_columns] = df_filtered_next[date_columns].replace('0', np.nan)

for col in date_columns:
    df_filtered_next[col] = df_filtered_next[col].apply(parse_dates)
    
# Fill NaN values with a placeholder date for aggregation purposes
placeholder_date = '2027-12-31'
df_filtered_next[date_columns] = df_filtered_next[date_columns].fillna(placeholder_date)
    
# Convert date columns to datetime
df_filtered_next[date_columns] = df_filtered_next[date_columns].apply(pd.to_datetime, format='%Y-%m-%d')
    
df_new = df_filtered_next.groupby(['Categories','Item #', 'Group Number','Additional Component','Vendor #','Check True/False','Arcadia ETD Smartsheet','EC ETD Smartsheet','WC ETD Smartsheet']).agg({
    'Whse': lambda x: set(x),
    'Arcadia ETD System Final': 'min',
    'EC ETD System Final': 'min',
    'WC ETD System Final': 'min'
}).reset_index()

# turn from datetime to string 
df_new = df_new[['Categories','Item #','Whse','Vendor #','Group Number','Additional Component','Arcadia ETD System Final','EC ETD System Final','WC ETD System Final','Arcadia ETD Smartsheet','EC ETD Smartsheet','WC ETD Smartsheet','Check True/False']]

for col in date_columns:
    df_new[col] = df_new[col].dt.date

existing_columns = [col for col in date_columns if col in df_new.columns]

if existing_columns:
    df_new[existing_columns] = df_new[existing_columns].astype(str)
else:
    print("No valid columns found for conversion.")

df_new = df_new.replace('2027-12-31', '0', inplace=False)

/var/folders/c7/zh_6w0ts24d8rh3tsybfhrz80000gn/T/ipykernel_34994/2550376722.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_next[date_columns] = df_filtered_next[date_columns].astype(str)
/var/folders/c7/zh_6w0ts24d8rh3tsybfhrz80000gn/T/ipykernel_34994/2550376722.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_next[date_columns] = df_filtered_next[date_columns].replace('0', np.nan)
/var/folders/c7/zh_6w0ts24d8rh3tsybfhrz80000gn/T/ipykernel_34994/2550376722.py:11: SettingWithC

In [734]:
df_new

,Categories,Item #,Whse,Vendor #,Group Number,Additional Component,Arcadia ETD System Final,EC ETD System Final,WC ETD System Final,Arcadia ETD Smartsheet,EC ETD Smartsheet,WC ETD Smartsheet,Check True/False
0,Accessories,A1001080,{5},613923.0,A1001080,None,0,0,2024-08-03,0,0,2024-08-03,True
1,Accessories,A1001081,{5},613923.0,A1001081,None,0,0,2024-09-07,0,0,2024-08-31,False
2,Accessories,A1001082,{5},613923.0,A1001082,None,0,0,2024-08-03,0,0,2024-08-03,True
3,Accessories,A4000669,{15},646821.0,A4000669,None,0,2024-08-10,0,0,2024-08-03,0,False
4,Accessories,A4000669,{1},646821.0,A4000669,None,2024-08-10,0,0,2024-08-10,0,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,Wall Unit,W100-101,"{1, 5, 17, 28, ECR, 15}",606629.0,W100,None,2024-07-27,2024-07-27,2024-08-03,0,0,0,False
392,Wall Unit,W100-12,"{1, 5, 17, 28, ECR, 15}",606629.0,W100,None,2024-07-27,2024-07-27,2024-08-03,0,0,0,False
393,Wall Unit,W100-121,"{1, 5, 17, 28, ECR, 15}",606629.0,W100,None,2024-08-10,2024-07-27,2024-08-03,0,0,0,False
394,Wall Unit,W100-13,"{1, 5, 17, 28, ECR, 15}",606629.0,W100,None,2024-09-07,2024-07-27,2024-08-03,0,0,0,False


In [735]:
df_new_1 = df_new

In [736]:
date_columns_1 = ['Arcadia ETD System Final','EC ETD System Final','WC ETD System Final','Arcadia ETD Smartsheet','EC ETD Smartsheet','WC ETD Smartsheet']

In [737]:
df_new_1[date_columns_1] = df_new_1[date_columns_1].astype(str)
df_new_1[date_columns_1] = df_new_1[date_columns_1].replace('0',np.nan)
place_nan = '1999-01-01'
df_new_1[date_columns_1] = df_new_1[date_columns_1].fillna(place_nan)
df_new_1[date_columns_1] = df_new_1[date_columns_1].apply(pd.to_datetime, format='%Y-%m-%d')

In [738]:
df_new_1 = df_new_1[['Categories','Item #','Vendor #','Group Number','Additional Component','Arcadia ETD System Final','EC ETD System Final','WC ETD System Final','Arcadia ETD Smartsheet','EC ETD Smartsheet','WC ETD Smartsheet']]


In [739]:
df_new_1 = df_new_1.groupby(['Categories', 'Item #','Vendor #','Group Number','Additional Component'])[date_columns_1].max().reset_index()

In [740]:
#Nếu Date đi Arca < Date đi EC thì highlight lên giúp t

In [741]:
df_new_1

,Categories,Item #,Vendor #,Group Number,Additional Component,Arcadia ETD System Final,EC ETD System Final,WC ETD System Final,Arcadia ETD Smartsheet,EC ETD Smartsheet,WC ETD Smartsheet
0,Accessories,A1001080,613923.0,A1001080,None,1999-01-01,1999-01-01,2024-08-03,1999-01-01,1999-01-01,2024-08-03
1,Accessories,A1001081,613923.0,A1001081,None,1999-01-01,1999-01-01,2024-09-07,1999-01-01,1999-01-01,2024-08-31
2,Accessories,A1001082,613923.0,A1001082,None,1999-01-01,1999-01-01,2024-08-03,1999-01-01,1999-01-01,2024-08-03
3,Accessories,A4000669,646821.0,A4000669,None,2024-08-10,2024-08-10,2024-08-17,2024-08-10,2024-08-03,2024-08-17
4,Accessories,A8000426,637187.0,A8000426,None,2024-08-03,2024-08-03,2024-08-17,2024-08-03,2024-08-03,2024-08-17
...,...,...,...,...,...,...,...,...,...,...,...
167,Wall Unit,W100-101,606629.0,W100,None,2024-07-27,2024-07-27,2024-08-03,1999-01-01,1999-01-01,1999-01-01
168,Wall Unit,W100-12,606629.0,W100,None,2024-07-27,2024-07-27,2024-08-03,1999-01-01,1999-01-01,1999-01-01
169,Wall Unit,W100-121,606629.0,W100,None,2024-08-10,2024-07-27,2024-08-03,1999-01-01,1999-01-01,1999-01-01
170,Wall Unit,W100-13,606629.0,W100,None,2024-09-07,2024-07-27,2024-08-03,1999-01-01,1999-01-01,1999-01-01


In [742]:
styled_df = highlight_condition_Arca_EC(df_new_1, 'Arcadia ETD System Final', 'EC ETD System Final',date_columns_1)

In [743]:
styled_df

,Categories,Item #,Vendor #,Group Number,Additional Component,Arcadia ETD System Final,EC ETD System Final,WC ETD System Final,Arcadia ETD Smartsheet,EC ETD Smartsheet,WC ETD Smartsheet
0,Accessories,A1001080,613923,A1001080,None,0,0,2024-08-03,0,0,2024-08-03
1,Accessories,A1001081,613923,A1001081,None,0,0,2024-09-07,0,0,2024-08-31
2,Accessories,A1001082,613923,A1001082,None,0,0,2024-08-03,0,0,2024-08-03
3,Accessories,A4000669,646821,A4000669,None,2024-08-10,2024-08-10,2024-08-17,2024-08-10,2024-08-03,2024-08-17
4,Accessories,A8000426,637187,A8000426,None,2024-08-03,2024-08-03,2024-08-17,2024-08-03,2024-08-03,2024-08-17
5,Accessories,A8000428,637187,A8000428,None,2024-08-03,2024-08-03,2024-08-17,2024-08-03,2024-08-03,2024-08-17
6,Accessories,A8000431,637187,A8000431,None,2024-08-03,2024-08-03,2024-08-17,2024-08-03,2024-08-03,2024-08-17
7,Accessories,A8010383,622820,A8010383,None,2024-08-31,2024-08-24,2024-09-07,2024-08-24,2024-08-24,2024-09-07
8,Accessories,A8010384,622820,A8010384,None,2024-08-31,2024-08-24,2024-09-07,2024-08-24,2024-08-24,2024-09-07
9,Bedroom,B895-31,653746,B895,None,2024-08-24,2024-08-24,2024-09-07,2024-08-17,2024-08-17,2024-08-31
